In [39]:
import pandas as pd
import numpy as np
import pyproj
import folium

In [2]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude","timezone", "dst", "tz", "type", "source"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["airline", "airline_id", "source", "source_id", "dest", "dest_id", "codeshare", "stops", "equipment"]

In [3]:
# Select the LEX routes, then join the source airports
lex_routes = routes[(routes['source']=="LEX")]
lex_routes = pd.merge(lex_routes, airports, left_on='source_id', right_on='id', how='left')

In [17]:
l = list(lex_routes.dest) #4 and 5 still show up as atlanta
l[0]

'ATL'

In [18]:
t = str(l[0])
df = routes[(routes['source'] == t)]

In [19]:
routes1 = pd.DataFrame(columns=['airline', 'airline_id', 'source', 'source_id', 'dest','dest_id','codeshare','stops','equipment'])
for i in range(len(l)):
   
    routess = routes[(routes['source']== str(l[i]))]
    routes1 = routes1.append(routess)
    #routes1 = pd.merge(lex_routes, airports, left_on='source_id', right_on='id', how='left')

In [20]:
len(routes1)

8189

In [23]:
dest1 = routes1.dest.drop_duplicates()
d = list(dest1)

In [24]:
routes2 = pd.DataFrame(columns=['airline', 'airline_id', 'source', 'source_id', 'dest','dest_id','codeshare','stops','equipment'])
for i in range(len(d)):
    routess = routes[(routes['source']== str(d[i]))]
   
    routes2 = routes2.append(routess)

In [27]:
dest2 = routes2.dest.drop_duplicates()
len(dest2)

1853

In [40]:
l = list(d)
route = pd.DataFrame(columns = routes.columns)
L = set(d)
while len(airports) != len(L):
    for i in range(len(l)):
        print(i)
        print(l[i])
        routess = routes[(routes['source']==str(l[i]))]
        route = route.append(routess)
        l = list(route.dest.unique)
        L = L.add()
        

0
LWB


TypeError: 'method' object is not iterable

In [41]:
L = set(['LEX'])
print(L)

L = L.add(set(['LEX']))

print(L)

{'LEX'}


TypeError: unhashable type: 'set'

In [42]:
s1 = list(dest1)
len(s1)

399

In [43]:
routes1_comb = pd.DataFrame(columns=['id', 'name', 'city', 'country', 'iata','icao','latitude','longitude','altitude', 'timezone','dst','tz','type','source'])
for i in range(len(s1)):

    routess = airports[(airports['iata']== str(s1[i]))]
    routes1_comb = routes1_comb.append(routess)

In [44]:
len(routes1_comb)

398

In [45]:
s2 = list(dest2)
len(s2)

1853

In [46]:
routes2_comb = pd.DataFrame(columns=['id', 'name', 'city', 'country', 'iata','icao','latitude','longitude','altitude', 'timezone','dst','tz','type','source'])
for i in range(len(s2)):

    routess = airports[(airports['iata']== str(s2[i]))]
    routes2_comb = routes2_comb.append(routess)

In [47]:
print(len(routes2_comb))
routes2_comb.head()

1808


,id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,tz,type,source
3467,3682.0,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,ATL,KATL,33.636700,-84.428101,1026.0,-5.0,A,America/New_York,airport,OurAirports
3498,3714.0,Washington Dulles International Airport,Washington,United States,IAD,KIAD,38.944500,-77.455803,312.0,-5.0,A,America/New_York,airport,OurAirports
3658,3878.0,Orlando International Airport,Orlando,United States,MCO,KMCO,28.429399,-81.308998,96.0,-5.0,A,America/New_York,airport,OurAirports
4739,6130.0,Mid Delta Regional Airport,Greenville,United States,GLH,KGLH,33.482899,-90.985603,131.0,-6.0,A,America/Chicago,airport,OurAirports
3611,3830.0,Chicago O'Hare International Airport,Chicago,United States,ORD,KORD,41.978600,-87.904800,672.0,-6.0,A,America/Chicago,airport,OurAirports


In [48]:
# create a basic map, centered on Lexington
lex_air = folium.Map(
    location=[38.034,-84.500],
    tiles='Stamen Toner',
    zoom_start=4)

In [49]:
# Define some empty sets
airport_set = set()
route_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in routes1_comb.iterrows():
    
    if row['iata'] not in airport_set: 
        popup_string = row['city'] + ' (' + row['iata'] + ')'
        marker = folium.CircleMarker([row["latitude"], row["longitude"]], 
                                     color='DarkCyan',
                                     fill_color='DarkCyan', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['source'])
        
   
        
lex_air

In [50]:
lex_air.save("lex_air.html")

In [51]:
# create a basic map, centered on Lexington
lex_air2 = folium.Map(
    location=[38.034,-84.500],
    tiles='Stamen Toner',
    zoom_start=4)

In [52]:
# Define some empty sets
airport_set = set()
route_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in routes2_comb.iterrows():
    
    if row['iata'] not in airport_set: 
        popup_string = row['city'] + ' (' + row['iata'] + ')'
        marker = folium.CircleMarker([row["latitude"], row["longitude"]], 
                                     color='DarkCyan',
                                     fill_color='DarkCyan', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air2)
        airport_set.add(row['source'])
        
   
        
lex_air2

In [53]:
lex_air2.save("lex_air2.html")

In [54]:
print(len(routes1_comb))
len(routes2_comb)

398


1808

In [56]:
iterrows?

Object `iterrows` not found.
